In [2]:
# Manejar el tiempo y contar el tiempo de ejecución
import time
comienzo = time.perf_counter()

# Convertir los datos en DataFrame y transformar
import pandas as pd

# Webscrapping
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# from selenium.webdriver.common.keys import Keys
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

try:
    # Ingresamos a la web de ambito financiero
    driver = webdriver.Chrome("chromedriver", options=chrome_options)
    url = "https://www.ambito.com/contenidos/riesgo-pais-historico.html"
    driver.get(url)

    # Cliquear en donde filtramos fecha de inicio

    filtro_fecha = driver.find_element(
        "xpath", "/html/body/main/div/div[1]/div[10]/div[1]/div[1]/input"
    )
    filtro_fecha.click()

    # Limpiamos y ponemos una fecha bien vieja para que nos dé el máximo que pueda
    filtro_fecha.clear()
    filtro_fecha.send_keys("05/06/1995")

    time.sleep(0.5)

    # Ejecutar el botón para filtrar la tabla
    boton_filtrar = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            ("xpath", "/html/body/main/div/div[1]/div[10]/div[1]/button")
        )
    )
    driver.execute_script("arguments[0].click();", boton_filtrar)

    # Esperar a que la tabla cargue, sins wait, sino toma los datos antes de que termine de cargar
    time.sleep(1.25)

    tabla = driver.find_element("xpath", "/html/body/main/div/div[1]/div[10]/table")

    # Tomamos el contenido HTML de la tabla
    tabla = tabla.get_attribute("outerHTML")

    # Desde el HTML creamos el dataframe
    riesgo_pais_df = pd.read_html(tabla)[0]
    riesgo_pais_df["Fecha"] = pd.to_datetime(
        riesgo_pais_df["Fecha"], format="%d-%m-%Y"
    ).dt.strftime("%d/%m/%y")
    riesgo_pais_df["Puntos"] = riesgo_pais_df["Puntos"] // 100

# Cerramos el navegador
finally:
    driver.quit()

# Exportamos a Excel
riesgo_pais_df.to_excel("Riesgo País.xlsx")

print(
    f"""
La duración de la ejecución total fue de: {time.perf_counter() - comienzo:.2f} segundos.
{riesgo_pais_df}""")


La duración de la ejecución total fue de: 12.08 segundos.
         Fecha  Puntos
0     19/06/23    2371
1     16/06/23    2371
2     15/06/23    2359
3     14/06/23    2330
4     13/06/23    2311
...        ...     ...
6322  17/12/98     758
6323  16/12/98     760
6324  15/12/98     734
6325  14/12/98     766
6326  11/12/98     720

[6327 rows x 2 columns]
